In [1]:
import requests
from lxml import etree
import numpy as np
import pandas as pd
import time 

In [2]:
# 获取爬取的城市和爬取的页码数
def get_city_page():
    city = input('请输入要爬取的城市：')
    page = int(input('请输入要爬取的页码数：'))
    return city,page

In [35]:
# 构建url_list
def create_url_lis(city,page):
    # 城市对应编码
    city_code_dict = {
        '上海':538, '北京':530, '广州':763, '深圳':765, '天津':531, '武汉':736, '西安':854, 
        '成都':801, '南京':635, '杭州':653, '重庆':551, '厦门':682,'济南':702, '青岛':703, '郑州':719
    }
    # 定义一个空列表用于存放url
    url_lis = []
    # 根据城市中文名取出对应的城市编码
    city_code = city_code_dict[city]
    # 循环遍历页码数，生成url_lis
    for p in range(page):
        url = 'https://sou.zhaopin.com/?jl={}&kw={}&p={}'.format(city_code,'数据分析',p+1)
        url_lis.append(url)
    return url_lis

In [4]:
# 以下代码为测试用
city,page = get_city_page()
url_lis = create_url_lis(city,page)
url_lis

请输入要爬取的城市：北京
请输入要爬取的页码数：1


['https://sou.zhaopin.com/?jl=530&kw=数据分析&p=1']

In [5]:
# 根据url获取网页源代码
def get_html(url):
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.70',
        'cookie': 'x-zp-client-id=373889a6-aab3-4eb9-813f-5ac0f06913fc; FSSBBIl1UgzbN7NO=5jB1BXq8Cik69q67XTB0xAGr3udQBbEx6eE3qmI_hl1ZPmIyHJrfpGPW1n3e4hkEDBGA3TZ291IGtpbqZ9b6ZFa; _uab_collina=166800275022344594750875; login-type=b; x-zp-device-id=7f1606ab4ce001192771371b516a2ab0; zp_passport_deepknow_sessionId=8f984297s16e6843ac9e58da24af56d4973b; at=2d4cb15f162249b29deb7c065e7cf535; rt=d4862c038e0e48b79c0d5b31c25f80e0; sts_deviceid=185fcbd18271e7-0217e2b4a5b5ca-7d5d5474-1327104-185fcbd18281f5; sts_sg=1; sts_evtseq=1; sts_sid=185fcbd182c23b-0166e48ecf8d2b-7d5d5474-1327104-185fcbd182f2ed; sts_chnlsid=Unknown; zp_src_url=https%3A%2F%2Fpassport.zhaopin.com%2F; ZP_OLD_FLAG=false; Hm_lvt_38ba284938d5eddca645bb5e02a02006=1674982530; acw_tc=276077cb16749836820493615ebe7f52ef6a41480bdc804e0e89c0c4f4f1a8; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%221152342112%22%2C%22first_id%22%3A%221845cb61e9e3bf-065609215c5b4d8-7d5d5474-1327104-1845cb61e9fb69%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E7%9B%B4%E6%8E%A5%E6%B5%81%E9%87%8F%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC_%E7%9B%B4%E6%8E%A5%E6%89%93%E5%BC%80%22%2C%22%24latest_referrer%22%3A%22%22%7D%2C%22identities%22%3A%22eyIkaWRlbnRpdHlfY29va2llX2lkIjoiMTg0NWNiNjFlOWUzYmYtMDY1NjA5MjE1YzViNGQ4LTdkNWQ1NDc0LTEzMjcxMDQtMTg0NWNiNjFlOWZiNjkiLCIkaWRlbnRpdHlfbG9naW5faWQiOiIxMTUyMzQyMTEyIn0%3D%22%2C%22history_login_id%22%3A%7B%22name%22%3A%22%24identity_login_id%22%2C%22value%22%3A%221152342112%22%7D%2C%22%24device_id%22%3A%221845cb61e9e3bf-065609215c5b4d8-7d5d5474-1327104-1845cb61e9fb69%22%7D; locationInfo_search={%22code%22:%22710%22%2C%22name%22:%22%E6%B3%B0%E5%AE%89%22%2C%22message%22:%22%E5%8C%B9%E9%85%8D%E5%88%B0%E5%B8%82%E7%BA%A7%E7%BC%96%E7%A0%81%22}; Hm_lpvt_38ba284938d5eddca645bb5e02a02006=1674983685; ssxmod_itna=Yqfx0iit37qxuDl4iqYI7pxUOIDBlAp=77e=Gx0HPreGzDAxn40iDtPoNgObF7BrTfqb7ZGr3NQIj44b=e78YpEdDU4i8DCMx3NrDem=D5xGoDPxDeDAQK0kUxCKD9DWKDFbq03Dc=508DeIFEaxDUlGhK0KDTx017xGODi5w0eYDma5DRpKDea5D9hPDwaHwlxG7U0EkDm4kKDDzpW4=KBgOUS8zN3OpHfOv7UoFhRELElIC9pKDoEPEKR9RnavLv0nDTeraQexxLliPzm0eKCeyj72BpQGKrlGP80iZKWNK1vXxDDcPaoD; ssxmod_itna2=Yqfx0iit37qxuDl4iqYI7pxUOIDBlAp=77e=DnKSiceDsqNQDLlQQ=uth4QIhdGcDYIxxD==; ZL_REPORT_GLOBAL={%22jobs%22:{%22funczoneShare%22:%22dtl_best_for_you%22%2C%22recommandActionidShare%22:%22c5800697-9fa8-4e20-a27f-885df3f1df82-job%22}}; selectCity_search=530; FSSBBIl1UgzbN7NP=53e3yqbHKiAEqqqDmdJJlPGKjLjYkIQKLjNvqDxRAfyeLRGv5kgRYheUa5JgT1v8h4Cm68chJnQG5C1hKoPNMaVD8Oly.5mDhsiKTqB7Y5QwVjfelfF30y.puCzAu9jKSjv1QSJOMeUduVxZuc9ooB6ctq4Dcf.PoV8rodypDJUawHtkENMt869uAvxJZOIlh9MU4r7iYpl1x6cOg_C3cj23kZiMAQc7MdvBfV_nucEVsxAcxUWVxhumwPThQWI4c2z0Vc7Js506C_Jm3hGbx_lkmJspIL2fqn.DhC8SEP4pu0df0DSviKCjTAUu_smJX1lZXQ6RiToz6uti9X5tTq1'
    }
    html = requests.get(url,headers = headers).text
    return html

In [6]:
# 以下为测试代码
url = 'https://sou.zhaopin.com/?jl=531&kw=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&p=1'
res = get_html(url)
print(res[0:600])

<!DOCTYPE html>
<html>
<head>
<meta name="renderer" content="webkit">
<meta name=viewport content="width=device-width,initial-scale=1,minimum-scale=1,maximum-scale=1,user-scalable=no,viewport-fit=cover">
<meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1">
<title>招聘（求职）尽在智联招聘</title>
<meta http-equiv="Cache-Control" content="no-transform" />
<meta http-equiv="Cache-Control" content="no-siteapp" />
<meta name="description" content="智联招聘为您提供招聘信息,工作尽在智联招聘" />
<meta name="keywords" content="招聘,招聘信息" />
<link rel="shortcut icon" type="image/png" href="//common-bucket.zhaopin.cn/img/zhilia


In [7]:
# 解析网页源代码，提取想要的信息
def transform_html(response):
    """
    解析网页源代码，提取想要的信息，并返回信息的dataframe
    response：抓取到的网页源代码
    """
    # 职位名称、薪资、地区、工作经验、学历、职位类别、招聘人数、职位描述、岗位职责、职位福利、公司名称、公司类型、公司规模
    html = etree.HTML(response)
    # 获取工作名称
    job = html.xpath('//span[@class="iteminfo__line1__jobname__name"]/@title')
    # 获取薪资范围
    salary = html.xpath('//p[@class="iteminfo__line2__jobdesc__salary"]/text()')
    for i in range(len(salary)):
        salary[i] = salary[i].strip('\n').strip(' ').rstrip('\n')
    # 获取地区、经验、学历信息
    location,experience,education = ([] for i in range(3))
    require = html.xpath('//ul[@class="iteminfo__line2__jobdesc__demand"]')
    for req in require:
        try:
            loc = req.xpath('.//li[@class="iteminfo__line2__jobdesc__demand__item"]/text()')[0]
            location.append(loc)
        except:
            location.append(np.nan)
        try:
            exp = req.xpath('.//li[@class="iteminfo__line2__jobdesc__demand__item"]/text()')[1]
            experience.append(exp)
        except:
            experience.append(np.nan)
        try:
            edu = req.xpath('.//li[@class="iteminfo__line2__jobdesc__demand__item"]/text()')[2]
            education.append(edu)
        except:
            education.append(np.nan)

    # 获取职位标签
    job_tag = []
    job_tag_lis = html.xpath('//div[@class="iteminfo__line3__welfare"]')
    for tag in job_tag_lis:
        tag_info = tag.xpath('.//div[@class="iteminfo__line3__welfare__item"]/text()')
        tag_info = str(tag_info)
        job_tag.append(tag_info)

    # 获取公司名称
    company_name = html.xpath('//span[@class="iteminfo__line1__compname__name"]/text()')

    # 获取公司类型、公司规模
    company_type = []
    company_size = []
    company_detail = html.xpath('//div[@class="iteminfo__line2__compdesc"]')
    for company in company_detail:
        try:
            com_type = company.xpath('.//span[@class="iteminfo__line2__compdesc__item"]/text()')[0]
            company_type.append(com_type)
        except:
            company_type.append(np.nan)
        try:
            com_size = company.xpath('.//span[@class="iteminfo__line2__compdesc__item"]/text()')[1]
            company_size.append(com_size)
        except:
            company_size.append(np.nan)

    data_lis = [job,salary,location,experience,education,job_tag,company_name,company_type,company_size]
    
    # 爬取结果合成一个dataframe
    get_data = pd.DataFrame(columns = ['职位名称','薪资范围','地点','工作经验','学历要求','岗位标签','公司名称','公司类型','公司规模'])
    for col,data in zip(get_data.columns,data_lis):
        get_data[col] = data
    # 返回数据的dataframe
    return get_data

In [8]:
# 以下为测试用代码
get_data = transform_html(res)
get_data.head(5)

,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模
0,数据分析专员,6千-1.1万,天津-西青,3-5年,大专,['运营数据分析'],北京达佳互联信息技术有限公司,上市公司,10000人以上
1,数据分析师,8千-1.1万,天津-河东,不限,本科,['运营数据分析'],石家庄飞谷互联网信息服务有限公司,股份制企业,1000-9999人
2,数据分析岗,6千-9千,天津,3-5年,大专,"['Power BI', '销售数据分析']",爱玛科技集团股份有限公司,上市公司,10000人以上
3,数据分析师,6千-1.2万,天津-西青,不限,本科,['数据分析'],天津卓漫科技有限公司,民营,500-999人
4,大数据分析师,8千-1.2万,天津-河西,不限,大专,"['R语言', 'SPSS', '数据分析', '大数据分析']",北京中阔鼎盛科技有限公司,,20-99人


In [9]:
# 循环爬取每一页的数据，并且合成为一个dataframe
def concat_data(url_lis):
    # 定义字典储存dataframe
    final_df_dict = {}
    for url,num in zip(url_lis,range(len(url_lis))):
        try:
            print('开始爬取第{}页'.format(num+1))
            # 获取网页源代码
            response = get_html(url)
            # 解析网页源代码并且生成一个dataframe
            final_df = transform_html(response)
            # 将dataframe保存到字典里
            final_df_dict[num] = final_df
            print('第{}页爬取完成'.format(num+1))
            # 爬取完成后程序休眠10秒
            time.sleep(10)
        except:
            print('所有页码都爬取完成！总计爬取{}页'.format(num+1))
    concat_df = pd.concat(list(final_df_dict.values()),ignore_index = True)
    return concat_df

In [14]:
# 爬取结果保存到csv
def save_df(df,city):
    file_name = '{}招聘信息.csv'.format(city)
    path = r'E:\2022年数据分析招聘数据爬取\{}'.format(file_name)
    df.to_csv(path,encoding = 'utf-8',index = False)
    print('{}保存成功'.format(file_name))
    return 

In [15]:
# 调用主函数，执行上边所有函数
def main():
    city,page = get_city_page()
    url_lis = create_url_lis(city,page)
    concat_df = concat_data(url_lis)
    save_df(concat_df,city)
    return print('所有程序执行完毕')

In [36]:
main()

请输入要爬取的城市：郑州
请输入要爬取的页码数：8
开始爬取第1页
第1页爬取完成
开始爬取第2页
第2页爬取完成
开始爬取第3页
第3页爬取完成
开始爬取第4页
第4页爬取完成
开始爬取第5页
第5页爬取完成
开始爬取第6页
第6页爬取完成
开始爬取第7页
第7页爬取完成
开始爬取第8页
第8页爬取完成
郑州招聘信息.csv保存成功
所有程序执行完毕
